In [200]:
# Install missing plotting libraries
%pip install matplotlib seaborn

# Import relevant libaries
import pandas as pd
import os
from pathlib import Path

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [201]:
# Load the participants_rows.csv file
data_dir = Path('google_exports')
df_surveys_final = pd.read_csv(data_dir / 'df_surveys_with_timing_filtered.csv')

df_surveys_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 55 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   participant_id              52 non-null     object 
 1   background_submitted_at     52 non-null     object 
 2   q2_gender                   52 non-null     object 
 3   q1_age_group                52 non-null     object 
 4   q3_education                52 non-null     object 
 5   q5_nationality              52 non-null     object 
 6   q7_ai_familiarity           52 non-null     int64  
 7   q8_attention_check          52 non-null     int64  
 8   q10_additional_info         52 non-null     object 
 9   q4_employment_status        52 non-null     object 
 10  q6_country_residence        52 non-null     object 
 11  q9_ai_usage_frequency       52 non-null     object 
 12  q11_response                52 non-null     object 
 13  instruction_submitted_at    52 non-nu

## Data Cleaning and Inconsistencies Handling for surveys datasets

### Column q5_nationality

In [202]:
import re

# define google_exports_dir for this notebook
google_exports_dir = '/path/to/google_exports'

# target column (fall back to 6th column if named differently)
col = 'q5_nationality'
if col not in df_surveys_final.columns and df_surveys_final.shape[1] > 5:
    col = df_surveys_final.columns[5]

def _normalize_nationality_to_country(value: str) -> str:
    if pd.isna(value):
        return value
    s = str(value).strip()
    s_low = s.lower()

    # split multi-values on common separators (commas, slashes, plus, ampersand, hyphens, "and")
    parts = re.split(r'[,/;+&\-]|\band\b|\s\+\s', s_low)
    parts = [p.strip() for p in parts if p.strip()]

    # patterns mapping various forms to canonical country names
    patterns = [
        (r'\b(?:autrian|austrian|austria)\b', 'Austria'),
        (r'\b(?:grrek|greek|greece)\b', 'Greece'),
        (r'\b(?:french|franc(?:e|ais|aise|aises)|france)\b', 'France'),
        (r'\b(?:italian|italiana|italia|italy)\b', 'Italy'),
        (r'\b(?:uk|u\.k\.|british|scottish|england|english|britain|brit)\b', 'United Kingdom'),
        (r'\b(?:usa|u\.s\.a\.|us|u\.s\.|america|american|united states(?: of america)?)\b', 'United States'),
        (r'\b(?:swiss|switzerland|schweiz)\b', 'Switzerland'),
        (r'\b(?:german|germany|de\b|deutsch)\b', 'Germany'),
        (r'\b(?:vietnamese|vietnam|viet[\- ]?nam)\b', 'Vietnam'),
    ]

    mapped = []
    for p in parts:
        matched = False
        for patt, country in patterns:
            if re.search(patt, p):
                mapped.append(country)
                matched = True
                break
        if not matched:
            # fallback: keep a cleaned, title-cased token (likely a country name or adjective)
            cleaned = re.sub(r'[^a-zA-Z\s]', ' ', p).strip()
            if cleaned:
                mapped.append(cleaned.title())

    # deduplicate preserving order
    seen = []
    for m in mapped:
        if m not in seen:
            seen.append(m)

    if not seen:
        return s
    # join multiple distinct normalized countries
    return ' / '.join(seen)

# create country column
country_col = f'{col}_country'
df_surveys_final[country_col] = df_surveys_final[col].apply(_normalize_nationality_to_country)


In [203]:
# drop the two columns if present
cols_to_drop = [col, clean_col]
df_surveys_final.drop(columns=cols_to_drop, inplace=True, errors='ignore')

### Column q10_additional_info

In [204]:
df_surveys_final["q10_additional_info"]

0                          Google, Ebay, Etsy, ViaLibri
1                                                Google
2                                                Google
3                                                Google
4                      Google, tiktok, official website
5                                                Google
6                                                Google
7                                               Google 
8                                                Shoppe
9                                                Google
10                                               Google
11                                               Google
12                                               Google
13                                               Amazon
14                                       Amazon, Google
15                                               Google
16                                       rednote，Taobao
17                                              

In [205]:
import pandas as pd
import re

# Column name
col = "q10_additional_info"

# -----------------------------------------
# Helper functions
# -----------------------------------------

def clean_item(item):
    """Clean individual tool name based on your rules."""
    if not item or pd.isna(item):
        return None

    item = item.strip()

    # Standardize common misspellings
    corrections = {
        r"\bgoogle\b": "Google",
        r"\bamazon\b": "Amazon",
        r"\bshoppe\b": "Shopee",
        r"\bshopee\b": "Shopee",
        r"\bwalmart\b": "Walmart",
        r"\bshein\b": "Shein",
        r"\btiktok\b": "TikTok",
        r"chat ?gpt": "ChatGPT",
    }
    for pattern, replacement in corrections.items():
        item = re.sub(pattern, replacement, item, flags=re.IGNORECASE)

    # Standardize price comparison terms
    if re.search(r"price\s*compar", item, flags=re.IGNORECASE):
        return "Price comparison sites"

    # Capitalize non-brand general terms
    if item.isalpha() and item.lower() not in [
        "google", "amazon", "shopee", "etsy", "ebay", "vinted",
        "facebook", "tiktok", "walmart", "shein", "taobao"
    ]:
        item = item.capitalize()

    return item


def split_items(text):
    """Split a cell into separate cleaned items."""
    if pd.isna(text):
        return []

    # Split by commas, slashes, and/or "and", "or"
    parts = re.split(r"[,/]| and | or ", str(text), flags=re.IGNORECASE)

    cleaned = []
    for p in parts:
        p = clean_item(p)
        if p:
            cleaned.append(p)

    # Remove duplicates while keeping order
    unique = []
    for x in cleaned:
        if x not in unique:
            unique.append(x)

    return unique

# -----------------------------------------
# PROCESS THE COLUMN IN PLACE
# -----------------------------------------

# Create a list column with cleaned + split items
df_surveys_final["splitted"] = df_surveys_final[col].apply(split_items)

# Find max number of items in any row
max_len = df_surveys_final["splitted"].apply(len).max()

# Create q10_tool_1 ... q10_tool_n directly in df_surveys_final
for i in range(1, max_len + 1):
    df_surveys_final[f"q10_tool_{i}"] = df_surveys_final["splitted"].apply(
        lambda x: x[i-1] if len(x) >= i else None
    )

# Remove original and helper columns
df_surveys_final.drop(columns=[col, "splitted"], inplace=True)

# Updates to apply: index 16 -> q10_tool_1 "RedNote", q10_tool_2 "Taobao"
#                   index 29 -> q10_tool_1 "Amazon",  q10_tool_2 "Shein"
updates = {
    16: ("RedNote", "Taobao"),
    29: ("Amazon", "Shein"),
}

for idx, (t1, t2) in updates.items():
    expanded_df.at[idx, "q10_tool_1"] = t1
    expanded_df.at[idx, "q10_tool_2"] = t2

# If the original dataframe also contains these columns and matching indices, update it too
if "df_surveys_final" in globals():
    needed = {"q10_tool_1", "q10_tool_2"}
    if needed.issubset(set(df_surveys_final.columns)):
        for idx, (t1, t2) in updates.items():
            if idx in df_surveys_final.index:
                df_surveys_final.at[idx, "q10_tool_1"] = t1
                df_surveys_final.at[idx, "q10_tool_2"] = t2

In [206]:
df_surveys_final

,participant_id,background_submitted_at,q2_gender,q1_age_group,q3_education,q7_ai_familiarity,q8_attention_check,q4_employment_status,q6_country_residence,q9_ai_usage_frequency,...,q42_future_usage_feedback,session_start_time,session_end_time,session_duration_ms,record_created_at,q5_nationality_country,q10_tool_1,q10_tool_2,q10_tool_3,q10_tool_4
0,09e6255b-ed66-4eea-a314-104a08130ac0,2025-10-31 22:11:56.864775+00:00,female,35-44,doctorate,7,1,employed,UK,more-than-10,...,"I liked the more minimal interface, and I like...",2025-10-31 22:10:58.834+00,2025-10-31 22:18:19.43+00,440596.0,2025-10-31 22:18:19.588138+00,United Kingdom,Google,Ebay,Etsy,Vialibri
1,15ef74b6-a61a-474c-b855-696b20ce58fb,2025-10-24 14:12:35.527412+00:00,male,55-and-above,master,1,1,employed,Germany,0-times,...,Maybe,2025-10-24 14:11:51.631+00,2025-10-24 14:24:48.07+00,776439.0,2025-10-24 14:24:48.231503+00,Germany,Google,None,None,None
2,1f0df1be-a1ea-4080-90c3-230fe9e35174,2025-11-10 03:25:51.855943+00:00,female,18-24,bachelor,6,1,student,United States,more-than-10,...,"Yes, it showed me relevant results for what I ...",2025-11-10 03:25:14.283+00,2025-11-10 03:26:06.698+00,52415.0,2025-11-10 03:26:07.101629+00,United States,Google,None,None,None
3,2afa9961-1844-49e8-80fc-444466532f46,2025-10-24 17:08:04.105300+00:00,female,25-34,master,6,6,unemployed,Russia,0-times,...,"Yes, the tool was transparent and easy to use,...",2025-10-24 17:06:39.799+00,2025-10-24 17:10:49.029+00,249230.0,2025-10-24 17:10:49.328167+00,Russian,Google,None,None,None
4,2d8dd1db-9d38-49e0-bf03-5b4735523d27,2025-10-29 09:54:21.050268+00:00,female,18-24,high-school,7,4,student,Switzerland,more-than-10,...,"No, AI didn’t show the classical bar abovd",2025-10-29 09:52:50.793+00,2025-10-29 09:54:46.465+00,115672.0,2025-10-29 09:54:46.569185+00,Switzerland,Google,TikTok,official website,None
5,2e912156-c7b7-4268-8420-128a859c4876,2025-10-26 15:45:12.277452+00:00,female,25-34,master,3,3,employed,Germany,0-times,...,"Yes, easy to use",2025-10-26 15:44:29.794+00,2025-10-26 15:45:39.02+00,69226.0,2025-10-26 15:45:39.157648+00,Vietnam / Germany,Google,None,None,None
6,3409f5be-93f2-44f4-8edb-910e95126257,2025-11-10 07:23:32.649361+00:00,female,25-34,bachelor,5,1,employed,Uk,1-2-times,...,Its easy to use,2025-11-10 07:22:57.74+00,2025-11-10 07:23:56.719+00,58979.0,2025-11-10 07:23:56.945929+00,United Kingdom,Google,None,None,None
7,35f58cd5-b9a5-4132-a94d-8fcad2800a59,2025-10-24 19:13:03.589983+00:00,female,35-44,bachelor,6,1,student,Germany,more-than-10,...,I don’t think so. It is not visually appealing,2025-10-24 19:11:36.971+00,2025-10-24 19:14:27.424+00,170453.0,2025-10-24 19:14:27.541556+00,Kazakh,Google,None,None,None
8,3beee5db-499b-4741-b3ea-72c6f17ffb86,2025-10-24 08:55:18.954744+00:00,female,25-34,bachelor,6,1,employed,Vietnam,more-than-10,...,My experience with this version of Google Sear...,2025-10-24 08:47:58.199+00,2025-10-24 09:02:52.01+00,893811.0,2025-10-24 09:02:52.809048+00,Vietnam,Shopee,None,None,None
9,403feef5-647d-4a34-a15a-c4bdc29fb2c2,2025-11-09 14:56:32.243330+00:00,female,18-24,high-school,5,1,student,Latvia,0-times,...,"Yes, it works well for me",2025-11-09 14:41:44.625+00,2025-11-09 14:58:58.615+00,1033990.0,2025-11-09 14:58:58.735264+00,Latvian,Google,None,None,None


### Column 14: q12_smartphone_model

In [207]:
df_surveys_final["q12_smartphone_model"]

0                                           Fairphone 5
1                                  Xiaomi Redmi Note 14
2                                              iPhone17
3                                        iPhone 17 Pro 
4                                        Iphone 15 plus
5                                    Samsung galaxy s24
6                                             iphone 12
7                                     Apple iPhone 16e 
8                                              Iphone16
9                                 Samsung Galaxy S25 FE
10                                            iPhone 12
11                                        Iphone 17 pro
12                                  Samsung Galaxy S25 
13                                                  NUU
14                                      Google Pixel 9+
15                                            iPhone 17
16                                          iPhone16pro
17                                   Samsung Gal

In [208]:
import pandas as pd
import re

col = "q12_smartphone_model"

# -----------------------------------------
# 1. Standardization Helpers
# -----------------------------------------

def clean_text(text):
    if pd.isna(text):
        return None
    text = text.strip()
    text = re.sub(r",\s*\(", " (", text)     # fix commas before parentheses
    text = re.sub(r"\s+", " ", text)         # collapse multiple spaces
    return text


def extract_parenthesis_comment(text):
    match = re.search(r"\((.*?)\)", text)
    return f"({match.group(1).strip()})" if match else ""


def normalize_iphone(model):
    model = re.sub(r"iphone", "Apple iPhone", model, flags=re.IGNORECASE)

    # Normalize things like "Apple iPhone 17 pro max"
    return re.sub(
        r"apple iphone\s*(\d+)\s*(pro max|pro|plus|max|air|e)?",
        lambda m: "Apple iPhone " + m.group(1) + (" " + m.group(2).title() if m.group(2) else ""),
        model,
        flags=re.IGNORECASE
    ).strip()


def normalize_samsung(model):
    model = re.sub(r"(samsung\s*galaxy|galaxy|samsung)", "Samsung Galaxy", model, flags=re.IGNORECASE)

    # Normalize model variants (FE, Ultra, etc.)
    return re.sub(
        r"samsung galaxy\s*(s?\d+[a-z]*\s*(ultra|fe|max)?)",
        lambda m: "Samsung Galaxy " + m.group(1).upper().replace("ULTRA", "Ultra").replace("FE", "FE"),
        model,
        flags=re.IGNORECASE
    ).strip()


def normalize_pixel(model):
    return re.sub(r"(google pixel|pixel)", "Google Pixel", model, flags=re.IGNORECASE).strip()


def normalize_xiaomi(model):
    model = re.sub(r"\bxiaomi\b", "Xiaomi", model, flags=re.IGNORECASE)
    model = re.sub(r"\bredmi\b", "Redmi", model, flags=re.IGNORECASE)
    return model.strip()


def normalize_model(text):
    """Main standardization pipeline."""
    if pd.isna(text):
        return None

    original = clean_text(text)
    par_comment = extract_parenthesis_comment(original)

    # Remove parentheses before processing
    text = re.sub(r"\(.*?\)", "", original).strip()
    temp = text.lower()

    if "iphone" in temp or temp.startswith("apple"):
        text = normalize_iphone(text)
    elif "samsung" in temp or "galaxy" in temp:
        text = normalize_samsung(text)
    elif "pixel" in temp:
        text = normalize_pixel(text)
    elif "xiaomi" in temp or "redmi" in temp:
        text = normalize_xiaomi(text)

    # Fix standalone "Apple"
    if text.lower() == "apple":
        text = "Apple iPhone"

    # Add back cleaned comment
    if par_comment:
        par_comment = par_comment.replace("renewed on amazon", "Renewed").replace("refurbished", "Refurbished")
        text = f"{text} {par_comment}"

    return text.strip()


# -----------------------------------------
# 2. Extract Brand Only
# -----------------------------------------

def extract_brand(text):
    if pd.isna(text):
        return None

    t = text.lower().strip()

    brand_map = {
        r"\bapple\b": "Apple",
        r"\biphone\b": "Apple",
        r"\bsamsung\b": "Samsung",
        r"\bgalaxy\b": "Samsung",
        r"\bgoogle pixel\b": "Google",
        r"\bpixel\b": "Google",
        r"\bxiaomi\b": "Xiaomi",
        r"\bredmi\b": "Redmi",
        r"\brealme\b": "Realme",
        r"\bnuu\b": "NUU",
        r"\bsony\b": "Sony",
        r"\bmotorola\b": "Motorola",
        r"\boppo\b": "Oppo",
        r"\bvivo\b": "Vivo",
        r"\bhuawei\b": "Huawei",
        r"\boneplus\b": "OnePlus",
    }

    for pattern, brand in brand_map.items():
        if re.search(pattern, t, flags=re.IGNORECASE):
            return brand

    # fallback: first word
    return text.split()[0].capitalize()


# -----------------------------------------
# 3. Apply to df_surveys_final
# -----------------------------------------

# Standardize smartphone models
df_surveys_final[col] = df_surveys_final[col].apply(normalize_model)

# Extract brand-only column
df_surveys_final["q12_brand_only"] = df_surveys_final[col].apply(extract_brand)

# Insert brand column right after q12_smartphone_model
insert_pos = df_surveys_final.columns.get_loc(col) + 1
df_surveys_final.insert(insert_pos, "q12_brand_only", df_surveys_final.pop("q12_brand_only"))


In [209]:
df_surveys_final

,participant_id,background_submitted_at,q2_gender,q1_age_group,q3_education,q7_ai_familiarity,q8_attention_check,q4_employment_status,q6_country_residence,q9_ai_usage_frequency,...,q42_future_usage_feedback,session_start_time,session_end_time,session_duration_ms,record_created_at,q5_nationality_country,q10_tool_1,q10_tool_2,q10_tool_3,q10_tool_4
0,09e6255b-ed66-4eea-a314-104a08130ac0,2025-10-31 22:11:56.864775+00:00,female,35-44,doctorate,7,1,employed,UK,more-than-10,...,"I liked the more minimal interface, and I like...",2025-10-31 22:10:58.834+00,2025-10-31 22:18:19.43+00,440596.0,2025-10-31 22:18:19.588138+00,United Kingdom,Google,Ebay,Etsy,Vialibri
1,15ef74b6-a61a-474c-b855-696b20ce58fb,2025-10-24 14:12:35.527412+00:00,male,55-and-above,master,1,1,employed,Germany,0-times,...,Maybe,2025-10-24 14:11:51.631+00,2025-10-24 14:24:48.07+00,776439.0,2025-10-24 14:24:48.231503+00,Germany,Google,None,None,None
2,1f0df1be-a1ea-4080-90c3-230fe9e35174,2025-11-10 03:25:51.855943+00:00,female,18-24,bachelor,6,1,student,United States,more-than-10,...,"Yes, it showed me relevant results for what I ...",2025-11-10 03:25:14.283+00,2025-11-10 03:26:06.698+00,52415.0,2025-11-10 03:26:07.101629+00,United States,Google,None,None,None
3,2afa9961-1844-49e8-80fc-444466532f46,2025-10-24 17:08:04.105300+00:00,female,25-34,master,6,6,unemployed,Russia,0-times,...,"Yes, the tool was transparent and easy to use,...",2025-10-24 17:06:39.799+00,2025-10-24 17:10:49.029+00,249230.0,2025-10-24 17:10:49.328167+00,Russian,Google,None,None,None
4,2d8dd1db-9d38-49e0-bf03-5b4735523d27,2025-10-29 09:54:21.050268+00:00,female,18-24,high-school,7,4,student,Switzerland,more-than-10,...,"No, AI didn’t show the classical bar abovd",2025-10-29 09:52:50.793+00,2025-10-29 09:54:46.465+00,115672.0,2025-10-29 09:54:46.569185+00,Switzerland,Google,TikTok,official website,None
5,2e912156-c7b7-4268-8420-128a859c4876,2025-10-26 15:45:12.277452+00:00,female,25-34,master,3,3,employed,Germany,0-times,...,"Yes, easy to use",2025-10-26 15:44:29.794+00,2025-10-26 15:45:39.02+00,69226.0,2025-10-26 15:45:39.157648+00,Vietnam / Germany,Google,None,None,None
6,3409f5be-93f2-44f4-8edb-910e95126257,2025-11-10 07:23:32.649361+00:00,female,25-34,bachelor,5,1,employed,Uk,1-2-times,...,Its easy to use,2025-11-10 07:22:57.74+00,2025-11-10 07:23:56.719+00,58979.0,2025-11-10 07:23:56.945929+00,United Kingdom,Google,None,None,None
7,35f58cd5-b9a5-4132-a94d-8fcad2800a59,2025-10-24 19:13:03.589983+00:00,female,35-44,bachelor,6,1,student,Germany,more-than-10,...,I don’t think so. It is not visually appealing,2025-10-24 19:11:36.971+00,2025-10-24 19:14:27.424+00,170453.0,2025-10-24 19:14:27.541556+00,Kazakh,Google,None,None,None
8,3beee5db-499b-4741-b3ea-72c6f17ffb86,2025-10-24 08:55:18.954744+00:00,female,25-34,bachelor,6,1,employed,Vietnam,more-than-10,...,My experience with this version of Google Sear...,2025-10-24 08:47:58.199+00,2025-10-24 09:02:52.01+00,893811.0,2025-10-24 09:02:52.809048+00,Vietnam,Shopee,None,None,None
9,403feef5-647d-4a34-a15a-c4bdc29fb2c2,2025-11-09 14:56:32.243330+00:00,female,18-24,high-school,5,1,student,Latvia,0-times,...,"Yes, it works well for me",2025-11-09 14:41:44.625+00,2025-11-09 14:58:58.615+00,1033990.0,2025-11-09 14:58:58.735264+00,Latvian,Google,None,None,None


### Column 15: q13_storage_capacity

In [210]:
df_surveys_final['q13_storage_capacity']

0                                                 256gb
1                                                   256
2                                                   NaN
3                                                256 GB
4                                                 128GB
5                                                   NaN
6                                                 128gb
7                                                   128
8                                                125 GB
9                                                256 GB
10                                               256 GB
11                                                   64
12                                               128 GB
13                                               128 GB
14                                               128 GB
15                                                  NaN
16                                                256GB
17                                              

In [211]:
col = "q13_storage_capacity"   # your column name

def standardize_storage(value):
    if pd.isna(value):
        return None

    text = str(value).lower()

    # Remove irrelevant words but keep digits and units
    text = text.replace(",", " ")

    # Extract the first storage-like pattern:
    # - number + optional unit
    # - handles: 64, 128gb, 256 gb, 1tb, 512-gb, etc.
    match = re.search(r"(\d+)\s*(gb|tb)?", text, flags=re.IGNORECASE)

    if not match:
        return None

    num = match.group(1)
    unit = match.group(2)

    # If no unit → default to GB
    if unit is None:
        unit = "GB"

    # Normalize unit casing
    unit = unit.upper()

    # Final standardized output
    return f"{num} {unit}"


# Apply to the dataframe directly
df_surveys_final[col] = df_surveys_final[col].apply(standardize_storage)


### Column 16: q14_color

In [212]:
df_surveys_final['q14_color']

0           transparent
1                 Green
2                   NaN
3        Cosmic orange 
4                 Black
5                   NaN
6                 black
7                 Black
8                 White
9                  Grey
10                White
11               Orange
12                 Navy
13             Titanium
14                Black
15                  NaN
16                White
17                black
18    Color - Deep Blue
19                white
20                Black
21                  NaN
22                 Gold
23                Black
24                 Blue
25         I don't know
26                  NaN
27                 blue
28                 Blue
29                black
30                  NaN
31                White
32                  NaN
33               Black 
34                white
35                Black
36                black
37                  NaN
38                 pink
39                 Blue
40            Deep Blue
41              

In [213]:
col = "q14_color"   # your column name

def clean_and_split_colors(text):
    if pd.isna(text):
        return []

    text = str(text).strip()

    # -----------------------------------------
    # 1. Remove irrelevant phrases like "Color - "
    # -----------------------------------------
    text = re.sub(r"color[\s:-]*", "", text, flags=re.IGNORECASE)

    # -----------------------------------------
    # 2. Replace any version of "I don't know" with empty field
    # -----------------------------------------
    if re.search(r"i\s*don'?t\s*know", text, flags=re.IGNORECASE):
        return []   # return empty list → empty columns

    # -----------------------------------------
    # 3. Normalize separators
    # -----------------------------------------
    text = re.sub(r"[/&|-]+", ",", text)  # /, -, &, |
    text = re.sub(r"\band\b", ",", text, flags=re.IGNORECASE)

    # Split by comma
    parts = [p.strip() for p in text.split(",") if p.strip()]

    cleaned = []
    for p in parts:
        # Capitalize each word
        words = p.split()
        cleaned_words = [w.capitalize() for w in words]
        cleaned.append(" ".join(cleaned_words))

    return cleaned


# -----------------------------------------
# Create a list column
# -----------------------------------------
df_surveys_final["q14_split"] = df_surveys_final[col].apply(clean_and_split_colors)

# -----------------------------------------
# Find max number of colors
# -----------------------------------------
max_len = df_surveys_final["q14_split"].apply(len).max()

# -----------------------------------------
# Create q14_color_1 ... q14_color_n
# -----------------------------------------
for i in range(1, max_len + 1):
    df_surveys_final[f"q14_color_{i}"] = df_surveys_final["q14_split"].apply(
        lambda lst: lst[i-1] if len(lst) >= i else None
    )

# -----------------------------------------
# Remove helper column & overwrite main column with cleaned first entry
# -----------------------------------------
df_surveys_final.drop(columns=["q14_split"], inplace=True)

df_surveys_final[col] = df_surveys_final["q14_color_1"]



In [214]:
df_surveys_final

,participant_id,background_submitted_at,q2_gender,q1_age_group,q3_education,q7_ai_familiarity,q8_attention_check,q4_employment_status,q6_country_residence,q9_ai_usage_frequency,...,session_start_time,session_end_time,session_duration_ms,record_created_at,q5_nationality_country,q10_tool_1,q10_tool_2,q10_tool_3,q10_tool_4,q14_color_1
0,09e6255b-ed66-4eea-a314-104a08130ac0,2025-10-31 22:11:56.864775+00:00,female,35-44,doctorate,7,1,employed,UK,more-than-10,...,2025-10-31 22:10:58.834+00,2025-10-31 22:18:19.43+00,440596.0,2025-10-31 22:18:19.588138+00,United Kingdom,Google,Ebay,Etsy,Vialibri,Transparent
1,15ef74b6-a61a-474c-b855-696b20ce58fb,2025-10-24 14:12:35.527412+00:00,male,55-and-above,master,1,1,employed,Germany,0-times,...,2025-10-24 14:11:51.631+00,2025-10-24 14:24:48.07+00,776439.0,2025-10-24 14:24:48.231503+00,Germany,Google,None,None,None,Green
2,1f0df1be-a1ea-4080-90c3-230fe9e35174,2025-11-10 03:25:51.855943+00:00,female,18-24,bachelor,6,1,student,United States,more-than-10,...,2025-11-10 03:25:14.283+00,2025-11-10 03:26:06.698+00,52415.0,2025-11-10 03:26:07.101629+00,United States,Google,None,None,None,None
3,2afa9961-1844-49e8-80fc-444466532f46,2025-10-24 17:08:04.105300+00:00,female,25-34,master,6,6,unemployed,Russia,0-times,...,2025-10-24 17:06:39.799+00,2025-10-24 17:10:49.029+00,249230.0,2025-10-24 17:10:49.328167+00,Russian,Google,None,None,None,Cosmic Orange
4,2d8dd1db-9d38-49e0-bf03-5b4735523d27,2025-10-29 09:54:21.050268+00:00,female,18-24,high-school,7,4,student,Switzerland,more-than-10,...,2025-10-29 09:52:50.793+00,2025-10-29 09:54:46.465+00,115672.0,2025-10-29 09:54:46.569185+00,Switzerland,Google,TikTok,official website,None,Black
5,2e912156-c7b7-4268-8420-128a859c4876,2025-10-26 15:45:12.277452+00:00,female,25-34,master,3,3,employed,Germany,0-times,...,2025-10-26 15:44:29.794+00,2025-10-26 15:45:39.02+00,69226.0,2025-10-26 15:45:39.157648+00,Vietnam / Germany,Google,None,None,None,None
6,3409f5be-93f2-44f4-8edb-910e95126257,2025-11-10 07:23:32.649361+00:00,female,25-34,bachelor,5,1,employed,Uk,1-2-times,...,2025-11-10 07:22:57.74+00,2025-11-10 07:23:56.719+00,58979.0,2025-11-10 07:23:56.945929+00,United Kingdom,Google,None,None,None,Black
7,35f58cd5-b9a5-4132-a94d-8fcad2800a59,2025-10-24 19:13:03.589983+00:00,female,35-44,bachelor,6,1,student,Germany,more-than-10,...,2025-10-24 19:11:36.971+00,2025-10-24 19:14:27.424+00,170453.0,2025-10-24 19:14:27.541556+00,Kazakh,Google,None,None,None,Black
8,3beee5db-499b-4741-b3ea-72c6f17ffb86,2025-10-24 08:55:18.954744+00:00,female,25-34,bachelor,6,1,employed,Vietnam,more-than-10,...,2025-10-24 08:47:58.199+00,2025-10-24 09:02:52.01+00,893811.0,2025-10-24 09:02:52.809048+00,Vietnam,Shopee,None,None,None,White
9,403feef5-647d-4a34-a15a-c4bdc29fb2c2,2025-11-09 14:56:32.243330+00:00,female,18-24,high-school,5,1,student,Latvia,0-times,...,2025-11-09 14:41:44.625+00,2025-11-09 14:58:58.615+00,1033990.0,2025-11-09 14:58:58.735264+00,Latvian,Google,None,None,None,Grey


### Column 17: q15_lowest_price

In [215]:
df_surveys_final['q15_lowest_price']

0                                        520
1                                        285
2                                        799
3                                       1449
4                                      549.-
5                                    530 eur
6                                       £140
7                                       606€
8                                        599
9                                    €649,99
10                                      450 
11                                      1299
12                                       799
13                                   137.43 
14                                     439.6
15                                       829
16                                    960USD
17                                    239.99
18                                    €1399 
19                                      1349
20                                      1200
21                                       444
22        

In [216]:
df = df_surveys_final
old_col = "q15_lowest_price"

USD_TO_EUR = 0.866
GBP_TO_EUR = 1.17


def parse_price_to_eur(val):
    if pd.isna(val):
        return None

    text = str(val).strip()

    if re.search(r"did not find a price", text, flags=re.IGNORECASE):
        return None

    # Clean basic formatting
    clean = (
        text.replace("€", "")
            .replace("eur", "")
            .replace("euros", "")
            .replace("euro", "")
            .replace(",", ".")
            .strip()
    )

    # GBP detection
    if text.startswith("£"):
        try:
            num = float(clean)
            return round(num * GBP_TO_EUR, 2)
        except:
            return None

    # USD detection
    if text.startswith("$") or "usd" in text.lower():
        try:
            num = float(re.sub(r"[^\d.]", "", clean))
            return round(num * USD_TO_EUR, 2)
        except:
            return None

    # EUR or plain numeric
    number_only = re.sub(r"[^\d.]", "", clean)

    try:
        return round(float(number_only), 2)
    except:
        return None


def assign_price_range(eur_val):
    if eur_val is None:
        return None
    if eur_val < 150:
        return "Under €150"
    elif eur_val < 300:
        return "€150-299"
    elif eur_val < 450:
        return "€300-449"
    elif eur_val < 600:
        return "€450-599"
    elif eur_val < 800:
        return "€600-799"
    else:
        return "Over €800"


# -------------------------------
# Create new standardized columns
# -------------------------------

df["q15_lowest_price_eur"] = df[old_col].apply(parse_price_to_eur)

# Format EUR as decimal-comma style
df["q15_lowest_price_eur"] = df["q15_lowest_price_eur"].apply(
    lambda x: f"{x:.2f}".replace(".", ",") if pd.notna(x) else None
)

# Price range based on EUR numeric value
df["q15_lowest_price_range"] = df["q15_lowest_price_eur"].apply(
    lambda v: assign_price_range(float(v.replace(",", "."))) if v not in (None, "") else None
)


# -------------------------------
# Move new columns after q14_color
# -------------------------------

insert_pos = df.columns.get_loc("q14_color") + 1

for col in ["q15_lowest_price_range", "q15_lowest_price_eur"]:
    df.insert(insert_pos, col, df.pop(col))
    insert_pos += 1  # Insert next column right after the previous one


# -------------------------------
# Remove old price column
# -------------------------------
df.drop(columns=[old_col], inplace=True)


In [217]:
df_surveys_final

,participant_id,background_submitted_at,q2_gender,q1_age_group,q3_education,q7_ai_familiarity,q8_attention_check,q4_employment_status,q6_country_residence,q9_ai_usage_frequency,...,session_start_time,session_end_time,session_duration_ms,record_created_at,q5_nationality_country,q10_tool_1,q10_tool_2,q10_tool_3,q10_tool_4,q14_color_1
0,09e6255b-ed66-4eea-a314-104a08130ac0,2025-10-31 22:11:56.864775+00:00,female,35-44,doctorate,7,1,employed,UK,more-than-10,...,2025-10-31 22:10:58.834+00,2025-10-31 22:18:19.43+00,440596.0,2025-10-31 22:18:19.588138+00,United Kingdom,Google,Ebay,Etsy,Vialibri,Transparent
1,15ef74b6-a61a-474c-b855-696b20ce58fb,2025-10-24 14:12:35.527412+00:00,male,55-and-above,master,1,1,employed,Germany,0-times,...,2025-10-24 14:11:51.631+00,2025-10-24 14:24:48.07+00,776439.0,2025-10-24 14:24:48.231503+00,Germany,Google,None,None,None,Green
2,1f0df1be-a1ea-4080-90c3-230fe9e35174,2025-11-10 03:25:51.855943+00:00,female,18-24,bachelor,6,1,student,United States,more-than-10,...,2025-11-10 03:25:14.283+00,2025-11-10 03:26:06.698+00,52415.0,2025-11-10 03:26:07.101629+00,United States,Google,None,None,None,None
3,2afa9961-1844-49e8-80fc-444466532f46,2025-10-24 17:08:04.105300+00:00,female,25-34,master,6,6,unemployed,Russia,0-times,...,2025-10-24 17:06:39.799+00,2025-10-24 17:10:49.029+00,249230.0,2025-10-24 17:10:49.328167+00,Russian,Google,None,None,None,Cosmic Orange
4,2d8dd1db-9d38-49e0-bf03-5b4735523d27,2025-10-29 09:54:21.050268+00:00,female,18-24,high-school,7,4,student,Switzerland,more-than-10,...,2025-10-29 09:52:50.793+00,2025-10-29 09:54:46.465+00,115672.0,2025-10-29 09:54:46.569185+00,Switzerland,Google,TikTok,official website,None,Black
5,2e912156-c7b7-4268-8420-128a859c4876,2025-10-26 15:45:12.277452+00:00,female,25-34,master,3,3,employed,Germany,0-times,...,2025-10-26 15:44:29.794+00,2025-10-26 15:45:39.02+00,69226.0,2025-10-26 15:45:39.157648+00,Vietnam / Germany,Google,None,None,None,None
6,3409f5be-93f2-44f4-8edb-910e95126257,2025-11-10 07:23:32.649361+00:00,female,25-34,bachelor,5,1,employed,Uk,1-2-times,...,2025-11-10 07:22:57.74+00,2025-11-10 07:23:56.719+00,58979.0,2025-11-10 07:23:56.945929+00,United Kingdom,Google,None,None,None,Black
7,35f58cd5-b9a5-4132-a94d-8fcad2800a59,2025-10-24 19:13:03.589983+00:00,female,35-44,bachelor,6,1,student,Germany,more-than-10,...,2025-10-24 19:11:36.971+00,2025-10-24 19:14:27.424+00,170453.0,2025-10-24 19:14:27.541556+00,Kazakh,Google,None,None,None,Black
8,3beee5db-499b-4741-b3ea-72c6f17ffb86,2025-10-24 08:55:18.954744+00:00,female,25-34,bachelor,6,1,employed,Vietnam,more-than-10,...,2025-10-24 08:47:58.199+00,2025-10-24 09:02:52.01+00,893811.0,2025-10-24 09:02:52.809048+00,Vietnam,Shopee,None,None,None,White
9,403feef5-647d-4a34-a15a-c4bdc29fb2c2,2025-11-09 14:56:32.243330+00:00,female,18-24,high-school,5,1,student,Latvia,0-times,...,2025-11-09 14:41:44.625+00,2025-11-09 14:58:58.615+00,1033990.0,2025-11-09 14:58:58.735264+00,Latvian,Google,None,None,None,Grey


### Column 20: q18_smartphone_features

In [218]:
df_surveys_final['q18_smartphone_features']

0             ["other","display","camera"]
1                   ["battery","software"]
2       ["camera","performance","storage"]
3           ["battery","camera","display"]
4       ["battery","camera","performance"]
5       ["battery","camera","performance"]
6           ["battery","camera","display"]
7           ["storage","battery","camera"]
8     ["display","performance","software"]
9       ["camera","performance","storage"]
10      ["battery","camera","performance"]
11          ["battery","camera","storage"]
12                              ["camera"]
13          ["battery","camera","display"]
14     ["battery","storage","performance"]
15      ["battery","camera","performance"]
16      ["camera","performance","storage"]
17               ["battery","performance"]
18      ["camera","performance","display"]
19          ["battery","camera","display"]
20          ["battery","display","camera"]
21          ["storage","camera","battery"]
22     ["performance","battery","storage"]
23    ["bat

In [219]:
import ast
import pandas as pd

df = df_surveys_final
old_col = "q18_smartphone_features"   # <-- adjust if your column has a different name

def parse_feature_list(x):
    if pd.isna(x):
        return []
    try:
        lst = ast.literal_eval(x)
    except:
        # Fallback manual parsing
        x = x.strip().lstrip("[").rstrip("]")
        lst = [i.strip().strip('"').strip("'") for i in x.split(",") if i.strip()]

    # Capitalize first letter of each feature
    return [item.capitalize() for item in lst]


# -----------------------------
# Convert each row into a list
# -----------------------------
df["parsed_features"] = df[old_col].apply(parse_feature_list)

# -----------------------------
# Determine insertion location
# -----------------------------
insert_pos = df.columns.get_loc(old_col)

# -----------------------------
# Create new q18_feature_* columns (capitalized)
# -----------------------------
for i in range(1, 4):   # 3 features max
    df.insert(
        insert_pos + (i - 1),
        f"q18_feature_{i}",
        df["parsed_features"].apply(lambda lst: lst[i-1] if len(lst) >= i else None)
    )

# -----------------------------
# Remove old + helper column
# -----------------------------
df.drop(columns=[old_col, "parsed_features"], inplace=True)


In [220]:
df_surveys_final

,participant_id,background_submitted_at,q2_gender,q1_age_group,q3_education,q7_ai_familiarity,q8_attention_check,q4_employment_status,q6_country_residence,q9_ai_usage_frequency,...,session_start_time,session_end_time,session_duration_ms,record_created_at,q5_nationality_country,q10_tool_1,q10_tool_2,q10_tool_3,q10_tool_4,q14_color_1
0,09e6255b-ed66-4eea-a314-104a08130ac0,2025-10-31 22:11:56.864775+00:00,female,35-44,doctorate,7,1,employed,UK,more-than-10,...,2025-10-31 22:10:58.834+00,2025-10-31 22:18:19.43+00,440596.0,2025-10-31 22:18:19.588138+00,United Kingdom,Google,Ebay,Etsy,Vialibri,Transparent
1,15ef74b6-a61a-474c-b855-696b20ce58fb,2025-10-24 14:12:35.527412+00:00,male,55-and-above,master,1,1,employed,Germany,0-times,...,2025-10-24 14:11:51.631+00,2025-10-24 14:24:48.07+00,776439.0,2025-10-24 14:24:48.231503+00,Germany,Google,None,None,None,Green
2,1f0df1be-a1ea-4080-90c3-230fe9e35174,2025-11-10 03:25:51.855943+00:00,female,18-24,bachelor,6,1,student,United States,more-than-10,...,2025-11-10 03:25:14.283+00,2025-11-10 03:26:06.698+00,52415.0,2025-11-10 03:26:07.101629+00,United States,Google,None,None,None,None
3,2afa9961-1844-49e8-80fc-444466532f46,2025-10-24 17:08:04.105300+00:00,female,25-34,master,6,6,unemployed,Russia,0-times,...,2025-10-24 17:06:39.799+00,2025-10-24 17:10:49.029+00,249230.0,2025-10-24 17:10:49.328167+00,Russian,Google,None,None,None,Cosmic Orange
4,2d8dd1db-9d38-49e0-bf03-5b4735523d27,2025-10-29 09:54:21.050268+00:00,female,18-24,high-school,7,4,student,Switzerland,more-than-10,...,2025-10-29 09:52:50.793+00,2025-10-29 09:54:46.465+00,115672.0,2025-10-29 09:54:46.569185+00,Switzerland,Google,TikTok,official website,None,Black
5,2e912156-c7b7-4268-8420-128a859c4876,2025-10-26 15:45:12.277452+00:00,female,25-34,master,3,3,employed,Germany,0-times,...,2025-10-26 15:44:29.794+00,2025-10-26 15:45:39.02+00,69226.0,2025-10-26 15:45:39.157648+00,Vietnam / Germany,Google,None,None,None,None
6,3409f5be-93f2-44f4-8edb-910e95126257,2025-11-10 07:23:32.649361+00:00,female,25-34,bachelor,5,1,employed,Uk,1-2-times,...,2025-11-10 07:22:57.74+00,2025-11-10 07:23:56.719+00,58979.0,2025-11-10 07:23:56.945929+00,United Kingdom,Google,None,None,None,Black
7,35f58cd5-b9a5-4132-a94d-8fcad2800a59,2025-10-24 19:13:03.589983+00:00,female,35-44,bachelor,6,1,student,Germany,more-than-10,...,2025-10-24 19:11:36.971+00,2025-10-24 19:14:27.424+00,170453.0,2025-10-24 19:14:27.541556+00,Kazakh,Google,None,None,None,Black
8,3beee5db-499b-4741-b3ea-72c6f17ffb86,2025-10-24 08:55:18.954744+00:00,female,25-34,bachelor,6,1,employed,Vietnam,more-than-10,...,2025-10-24 08:47:58.199+00,2025-10-24 09:02:52.01+00,893811.0,2025-10-24 09:02:52.809048+00,Vietnam,Shopee,None,None,None,White
9,403feef5-647d-4a34-a15a-c4bdc29fb2c2,2025-11-09 14:56:32.243330+00:00,female,18-24,high-school,5,1,student,Latvia,0-times,...,2025-11-09 14:41:44.625+00,2025-11-09 14:58:58.615+00,1033990.0,2025-11-09 14:58:58.735264+00,Latvian,Google,None,None,None,Grey


### Column 46: q39_contradictory_handling

In [221]:
df_surveys_final['q39_contradictory_handling']

0                 ["additional_sources","own_judgment"]
1                                 ["no_contradictions"]
2                                ["additional_sources"]
3                      ["most_detailed","own_judgment"]
4                 ["additional_sources","own_judgment"]
5                 ["additional_sources","own_judgment"]
6                                      ["first_result"]
7                                 ["no_contradictions"]
8                 ["additional_sources","own_judgment"]
9                ["additional_sources","most_detailed"]
10    ["additional_sources","most_detailed","own_jud...
11                                     ["own_judgment"]
12                               ["additional_sources"]
13                      ["first_result","own_judgment"]
14                               ["additional_sources"]
15                                ["no_contradictions"]
16    ["additional_sources","own_judgment","most_det...
17                                ["no_contradic

In [223]:
df = df_surveys_final
old_col = "q39_contradictory_handling"   # <-- rename to your actual column name

def parse_list(x):
    """Parse list-like strings; capitalize items."""
    if pd.isna(x):
        return []
    try:
        lst = ast.literal_eval(x)
    except:
        # fallback parser
        x = x.strip().lstrip("[").rstrip("]")
        lst = [i.strip().strip('"').strip("'") for i in x.split(",") if i.strip()]
    # Capitalize first letter of each item
    return [item.capitalize() for item in lst]


# -----------------------------
# Convert each row into a list
# -----------------------------
df["parsed_temp"] = df[old_col].apply(parse_list)

# -----------------------------
# Determine max number of items
# -----------------------------
max_len = df["parsed_temp"].apply(len).max()

# -----------------------------
# Determine insertion location
# -----------------------------
insert_pos = df.columns.get_loc(old_col)

# -----------------------------
# Create q39_contradiction_* columns
# -----------------------------
for i in range(1, max_len + 1):
    df.insert(
        insert_pos + (i - 1),
        f"q39_contradiction_{i}",
        df["parsed_temp"].apply(lambda lst: lst[i-1] if len(lst) >= i else None)
    )

# -----------------------------
# Remove old + temporary column
# -----------------------------
df.drop(columns=[old_col, "parsed_temp"], inplace=True)


In [224]:
df_surveys_final

,participant_id,background_submitted_at,q2_gender,q1_age_group,q3_education,q7_ai_familiarity,q8_attention_check,q4_employment_status,q6_country_residence,q9_ai_usage_frequency,...,session_start_time,session_end_time,session_duration_ms,record_created_at,q5_nationality_country,q10_tool_1,q10_tool_2,q10_tool_3,q10_tool_4,q14_color_1
0,09e6255b-ed66-4eea-a314-104a08130ac0,2025-10-31 22:11:56.864775+00:00,female,35-44,doctorate,7,1,employed,UK,more-than-10,...,2025-10-31 22:10:58.834+00,2025-10-31 22:18:19.43+00,440596.0,2025-10-31 22:18:19.588138+00,United Kingdom,Google,Ebay,Etsy,Vialibri,Transparent
1,15ef74b6-a61a-474c-b855-696b20ce58fb,2025-10-24 14:12:35.527412+00:00,male,55-and-above,master,1,1,employed,Germany,0-times,...,2025-10-24 14:11:51.631+00,2025-10-24 14:24:48.07+00,776439.0,2025-10-24 14:24:48.231503+00,Germany,Google,None,None,None,Green
2,1f0df1be-a1ea-4080-90c3-230fe9e35174,2025-11-10 03:25:51.855943+00:00,female,18-24,bachelor,6,1,student,United States,more-than-10,...,2025-11-10 03:25:14.283+00,2025-11-10 03:26:06.698+00,52415.0,2025-11-10 03:26:07.101629+00,United States,Google,None,None,None,None
3,2afa9961-1844-49e8-80fc-444466532f46,2025-10-24 17:08:04.105300+00:00,female,25-34,master,6,6,unemployed,Russia,0-times,...,2025-10-24 17:06:39.799+00,2025-10-24 17:10:49.029+00,249230.0,2025-10-24 17:10:49.328167+00,Russian,Google,None,None,None,Cosmic Orange
4,2d8dd1db-9d38-49e0-bf03-5b4735523d27,2025-10-29 09:54:21.050268+00:00,female,18-24,high-school,7,4,student,Switzerland,more-than-10,...,2025-10-29 09:52:50.793+00,2025-10-29 09:54:46.465+00,115672.0,2025-10-29 09:54:46.569185+00,Switzerland,Google,TikTok,official website,None,Black
5,2e912156-c7b7-4268-8420-128a859c4876,2025-10-26 15:45:12.277452+00:00,female,25-34,master,3,3,employed,Germany,0-times,...,2025-10-26 15:44:29.794+00,2025-10-26 15:45:39.02+00,69226.0,2025-10-26 15:45:39.157648+00,Vietnam / Germany,Google,None,None,None,None
6,3409f5be-93f2-44f4-8edb-910e95126257,2025-11-10 07:23:32.649361+00:00,female,25-34,bachelor,5,1,employed,Uk,1-2-times,...,2025-11-10 07:22:57.74+00,2025-11-10 07:23:56.719+00,58979.0,2025-11-10 07:23:56.945929+00,United Kingdom,Google,None,None,None,Black
7,35f58cd5-b9a5-4132-a94d-8fcad2800a59,2025-10-24 19:13:03.589983+00:00,female,35-44,bachelor,6,1,student,Germany,more-than-10,...,2025-10-24 19:11:36.971+00,2025-10-24 19:14:27.424+00,170453.0,2025-10-24 19:14:27.541556+00,Kazakh,Google,None,None,None,Black
8,3beee5db-499b-4741-b3ea-72c6f17ffb86,2025-10-24 08:55:18.954744+00:00,female,25-34,bachelor,6,1,employed,Vietnam,more-than-10,...,2025-10-24 08:47:58.199+00,2025-10-24 09:02:52.01+00,893811.0,2025-10-24 09:02:52.809048+00,Vietnam,Shopee,None,None,None,White
9,403feef5-647d-4a34-a15a-c4bdc29fb2c2,2025-11-09 14:56:32.243330+00:00,female,18-24,high-school,5,1,student,Latvia,0-times,...,2025-11-09 14:41:44.625+00,2025-11-09 14:58:58.615+00,1033990.0,2025-11-09 14:58:58.735264+00,Latvian,Google,None,None,None,Grey
